# About this Notebook
---

**Bayesian Gaussian CP decomposition** (or **BGCP** for short) is a type of Bayesian tensor decomposition that achieves state-of-the-art results on challenging the missing data imputation problem. In the following, we will discuss:

- What the Bayesian Gaussian CP decomposition is.

- How to implement BGCP mainly using Python `numpy` with high efficiency.

- How to make imputations with real-world spatiotemporal datasets.

If you want to understand BGCP and its modeling tricks in detail, our paper is for you:

> Xinyu Chen, Zhaocheng He, Lijun Sun (2019). **A Bayesian tensor decomposition approach for spatiotemporal traffic data imputation**. Transportation Research Part C: Emerging Technologies, 98: 73-84.

## Quick Run

This notebook is publicly available at [https://github.com/xinychen/tensor-learning](https://github.com/xinychen/tensor-learning).

We start by importing the necessary dependencies. We will make use of `numpy` and `scipy`.

In [1]:
import numpy as np
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart
from numpy.linalg import inv as inv

## Matrix Computation Concepts

### 1) Kronecker product

- **Definition**:

Given two matrices $A\in\mathbb{R}^{m_1\times n_1}$ and $B\in\mathbb{R}^{m_2\times n_2}$, then, the **Kronecker product** between these two matrices is defined as

$$A\otimes B=\left[ \begin{array}{cccc} a_{11}B & a_{12}B & \cdots & a_{1m_2}B \\ a_{21}B & a_{22}B & \cdots & a_{2m_2}B \\ \vdots & \vdots & \ddots & \vdots \\ a_{m_11}B & a_{m_12}B & \cdots & a_{m_1m_2}B \\ \end{array} \right]$$
where the symbol $\otimes$ denotes Kronecker product, and the size of resulted $A\otimes B$ is $(m_1m_2)\times (n_1n_2)$ (i.e., $m_1\times m_2$ columns and $n_1\times n_2$ rows).

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]$ and $B=\left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10 \\ \end{array} \right]$, then, we have

$$A\otimes B=\left[ \begin{array}{cc} 1\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 2\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ 3\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 4\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cccccc} 5 & 6 & 7 & 10 & 12 & 14 \\ 8 & 9 & 10 & 16 & 18 & 20 \\ 15 & 18 & 21 & 20 & 24 & 28 \\ 24 & 27 & 30 & 32 & 36 & 40 \\ \end{array} \right]\in\mathbb{R}^{4\times 6}.$$


### 2) Khatri-Rao product (`kr_prod`)

- **Definition**:

Given two matrices $A=\left( \boldsymbol{a}_1,\boldsymbol{a}_2,...,\boldsymbol{a}_r \right)\in\mathbb{R}^{m\times r}$ and $B=\left( \boldsymbol{b}_1,\boldsymbol{b}_2,...,\boldsymbol{b}_r \right)\in\mathbb{R}^{n\times r}$ with same number of columns, then, the **Khatri-Rao product** (or **column-wise Kronecker product**) between $A$ and $B$ is given as follows,

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2,...,\boldsymbol{a}_r\otimes \boldsymbol{b}_r \right)\in\mathbb{R}^{(mn)\times r},$$
where the symbol $\odot$ denotes Khatri-Rao product, and $\otimes$ denotes Kronecker product.

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]=\left( \boldsymbol{a}_1,\boldsymbol{a}_2 \right) $ and $B=\left[ \begin{array}{cc} 5 & 6 \\ 7 & 8 \\ 9 & 10 \\ \end{array} \right]=\left( \boldsymbol{b}_1,\boldsymbol{b}_2 \right) $, then, we have

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2 \right) $$

$$=\left[ \begin{array}{cc} \left[ \begin{array}{c} 1 \\ 3 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 5 \\ 7 \\ 9 \\ \end{array} \right] & \left[ \begin{array}{c} 2 \\ 4 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 6 \\ 8 \\ 10 \\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cc} 5 & 12 \\ 7 & 16 \\ 9 & 20 \\ 15 & 24 \\ 21 & 32 \\ 27 & 40 \\ \end{array} \right]\in\mathbb{R}^{6\times 2}.$$

In [2]:
def kr_prod(a, b):
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)

In [3]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8], [9, 10]])
print(kr_prod(A, B))

[[ 5 12]
 [ 7 16]
 [ 9 20]
 [15 24]
 [21 32]
 [27 40]]


### 3) CP decomposition

#### CP Combination (`cp_combine`)

- **Definition**:

The CP decomposition factorizes a tensor into a sum of outer products of vectors. For example, for a third-order tensor $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$, the CP decomposition can be written as

$$\hat{\mathcal{Y}}=\sum_{s=1}^{r}\boldsymbol{u}_{s}\circ\boldsymbol{v}_{s}\circ\boldsymbol{x}_{s},$$
or element-wise,

$$\hat{y}_{ijt}=\sum_{s=1}^{r}u_{is}v_{js}x_{ts},\forall (i,j,t),$$
where vectors $\boldsymbol{u}_{s}\in\mathbb{R}^{m},\boldsymbol{v}_{s}\in\mathbb{R}^{n},\boldsymbol{x}_{s}\in\mathbb{R}^{f}$ are columns of factor matrices $U\in\mathbb{R}^{m\times r},V\in\mathbb{R}^{n\times r},X\in\mathbb{R}^{f\times r}$, respectively. The symbol $\circ$ denotes vector outer product.

- **Example**:

Given matrices $U=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]\in\mathbb{R}^{2\times 2}$, $V=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ 5 & 6 \\ \end{array} \right]\in\mathbb{R}^{3\times 2}$ and $X=\left[ \begin{array}{cc} 1 & 5 \\ 2 & 6 \\ 3 & 7 \\ 4 & 8 \\ \end{array} \right]\in\mathbb{R}^{4\times 2}$, then if $\hat{\mathcal{Y}}=\sum_{s=1}^{r}\boldsymbol{u}_{s}\circ\boldsymbol{v}_{s}\circ\boldsymbol{x}_{s}$, then, we have

$$\hat{Y}_1=\hat{\mathcal{Y}}(:,:,1)=\left[ \begin{array}{ccc} 31 & 42 & 65 \\ 63 & 86 & 135 \\ \end{array} \right],$$
$$\hat{Y}_2=\hat{\mathcal{Y}}(:,:,2)=\left[ \begin{array}{ccc} 38 & 52 & 82 \\ 78 & 108 & 174 \\ \end{array} \right],$$
$$\hat{Y}_3=\hat{\mathcal{Y}}(:,:,3)=\left[ \begin{array}{ccc} 45 & 62 & 99 \\ 93 & 130 & 213 \\ \end{array} \right],$$
$$\hat{Y}_4=\hat{\mathcal{Y}}(:,:,4)=\left[ \begin{array}{ccc} 52 & 72 & 116 \\ 108 & 152 & 252 \\ \end{array} \right].$$

In [4]:
def cp_combine(var):
    return np.einsum('is, js, ts -> ijt', var[0], var[1], var[2])

In [5]:
factor = [np.array([[1, 2], [3, 4]]), np.array([[1, 3], [2, 4], [5, 6]]), 
          np.array([[1, 5], [2, 6], [3, 7], [4, 8]])]
print(cp_combine(factor))
print()
print('tensor size:')
print(cp_combine(factor).shape)

[[[ 31  38  45  52]
  [ 42  52  62  72]
  [ 65  82  99 116]]

 [[ 63  78  93 108]
  [ 86 108 130 152]
  [135 174 213 252]]]

tensor size:
(2, 3, 4)


### 4) Tensor Unfolding (`ten2mat`)

Using numpy reshape to perform 3rd rank tensor unfold operation. [[**link**](https://stackoverflow.com/questions/49970141/using-numpy-reshape-to-perform-3rd-rank-tensor-unfold-operation)]

In [6]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [7]:
X = np.array([[[1, 2, 3, 4], [3, 4, 5, 6]], 
              [[5, 6, 7, 8], [7, 8, 9, 10]], 
              [[9, 10, 11, 12], [11, 12, 13, 14]]])
print('tensor size:')
print(X.shape)
print('original tensor:')
print(X)
print()
print('(1) mode-1 tensor unfolding:')
print(ten2mat(X, 0))
print()
print('(2) mode-2 tensor unfolding:')
print(ten2mat(X, 1))
print()
print('(3) mode-3 tensor unfolding:')
print(ten2mat(X, 2))

tensor size:
(3, 2, 4)
original tensor:
[[[ 1  2  3  4]
  [ 3  4  5  6]]

 [[ 5  6  7  8]
  [ 7  8  9 10]]

 [[ 9 10 11 12]
  [11 12 13 14]]]

(1) mode-1 tensor unfolding:
[[ 1  3  2  4  3  5  4  6]
 [ 5  7  6  8  7  9  8 10]
 [ 9 11 10 12 11 13 12 14]]

(2) mode-2 tensor unfolding:
[[ 1  5  9  2  6 10  3  7 11  4  8 12]
 [ 3  7 11  4  8 12  5  9 13  6 10 14]]

(3) mode-3 tensor unfolding:
[[ 1  5  9  3  7 11]
 [ 2  6 10  4  8 12]
 [ 3  7 11  5  9 13]
 [ 4  8 12  6 10 14]]


In [8]:
def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

### 5) Computing Covariance Matrix (`cov_mat`)

For any matrix $X\in\mathbb{R}^{m\times n}$, `cov_mat` can return a $n\times n$ covariance matrix for special use in the following.

In [9]:
def cov_mat(mat):
    dim1, dim2 = mat.shape
    new_mat = np.zeros((dim2, dim2))
    mat_bar = np.mean(mat, axis = 0)
    for i in range(dim1):
        new_mat += np.einsum('i, j -> ij', mat[i, :] - mat_bar, mat[i, :] - mat_bar)
    return new_mat

## Bayesian Gaussian CP decomposition (BGCP)

### 1) Model Description

#### Gaussian assumption

Given a matrix $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$ which suffers from missing values, then the factorization can be applied to reconstruct the missing values within $\mathcal{Y}$ by

$$y_{ijt}\sim\mathcal{N}\left(\sum_{s=1}^{r}u_{is} v_{js} x_{ts},\tau^{-1}\right),\forall (i,j,t),$$
where vectors $\boldsymbol{u}_{s}\in\mathbb{R}^{m},\boldsymbol{v}_{s}\in\mathbb{R}^{n},\boldsymbol{x}_{s}\in\mathbb{R}^{f}$ are columns of latent factor matrices, and $u_{is},v_{js},x_{ts}$ are their elements. The precision term $\tau$ is an inverse of Gaussian variance.

#### Bayesian framework

Based on the Gaussian assumption over tensor elements $y_{ijt},(i,j,t)\in\Omega$ (where $\Omega$ is a index set indicating observed tensor elements), the conjugate priors of model parameters (i.e., latent factors and precision term) and hyperparameters are given as

$$\boldsymbol{u}_{i}\sim\mathcal{N}\left(\boldsymbol{\mu}_{u},\Lambda_{u}^{-1}\right),\forall i,$$
$$\boldsymbol{v}_{j}\sim\mathcal{N}\left(\boldsymbol{\mu}_{v},\Lambda_{v}^{-1}\right),\forall j,$$
$$\boldsymbol{x}_{t}\sim\mathcal{N}\left(\boldsymbol{\mu}_{x},\Lambda_{x}^{-1}\right),\forall t,$$
$$\tau\sim\text{Gamma}\left(a_0,b_0\right),$$
$$\boldsymbol{\mu}_{u}\sim\mathcal{N}\left(\boldsymbol{\mu}_0,\left(\beta_0\Lambda_u\right)^{-1}\right),\Lambda_u\sim\mathcal{W}\left(W_0,\nu_0\right),$$
$$\boldsymbol{\mu}_{v}\sim\mathcal{N}\left(\boldsymbol{\mu}_0,\left(\beta_0\Lambda_v\right)^{-1}\right),\Lambda_v\sim\mathcal{W}\left(W_0,\nu_0\right),$$
$$\boldsymbol{\mu}_{x}\sim\mathcal{N}\left(\boldsymbol{\mu}_0,\left(\beta_0\Lambda_x\right)^{-1}\right),\Lambda_x\sim\mathcal{W}\left(W_0,\nu_0\right).$$


### 2) Posterior Inference

In the following, we will apply Gibbs sampling to implement our Bayesian inference for the matrix factorization task.

#### - Sampling latent factors $\boldsymbol{u}_{i},i\in\left\{1,2,...,m\right\}$

Draw $\boldsymbol{u}_{i}\sim\mathcal{N}\left(\boldsymbol{\mu}_i^{*},(\Lambda_{i}^{*})^{-1}\right)$ with following parameters:

$$\boldsymbol{\mu}_{i}^{*}=\left(\Lambda_{i}^{*}\right)^{-1}\left\{\tau\sum_{j,t:(i,j,t)\in\Omega}y_{ijt}\left(\boldsymbol{v}_{j}\circledast\boldsymbol{x}_{t}\right)+\Lambda_u\boldsymbol{\mu}_u\right\},$$

$$\Lambda_{i}^{*}=\tau\sum_{j,t:(i,j,t)\in\Omega}\left(\boldsymbol{v}_{j}\circledast\boldsymbol{x}_{t}\right)\left(\boldsymbol{v}_{j}\circledast\boldsymbol{x}_{t}\right)^{T}+\Lambda_u.$$


#### - Sampling latent factors $\boldsymbol{v}_{j},j\in\left\{1,2,...,n\right\}$

Draw $\boldsymbol{v}_{j}\sim\mathcal{N}\left(\boldsymbol{\mu}_j^{*},(\Lambda_{j}^{*})^{-1}\right)$ with following parameters:

$$\boldsymbol{\mu}_{j}^{*}=\left(\Lambda_{j}^{*}\right)^{-1}\left\{\tau\sum_{i,t:(i,j,t)\in\Omega}y_{ijt}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{x}_{t}\right)+\Lambda_v\boldsymbol{\mu}_v\right\}$$

$$\Lambda_{j}^{*}=\tau\sum_{i,t:(i,j,t)\in\Omega}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{x}_{t}\right)\left(\boldsymbol{u}_{i}\circledast\boldsymbol{x}_{t}\right)^{T}+\Lambda_v.$$


#### - Sampling latent factors $\boldsymbol{x}_{t},t\in\left\{1,2,...,f\right\}$

Draw $\boldsymbol{x}_{t}\sim\mathcal{N}\left(\boldsymbol{\mu}_t^{*},(\Lambda_{t}^{*})^{-1}\right)$ with following parameters:

$$\boldsymbol{\mu}_{t}^{*}=\left(\Lambda_{t}^{*}\right)^{-1}\left\{\tau\sum_{i,j:(i,j,t)\in\Omega}y_{ijt}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{v}_{j}\right)+\Lambda_x\boldsymbol{\mu}_x\right\}$$

$$\Lambda_{t}^{*}=\tau\sum_{i,j:(i,j,t)\in\Omega}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{v}_{j}\right)\left(\boldsymbol{u}_{i}\circledast\boldsymbol{v}_{j}\right)^{T}+\Lambda_x.$$


#### - Sampling precision term $\tau$

Draw $\tau\in\text{Gamma}\left(a^{*},b^{*}\right)$ with following parameters:

$$a^{*}=a_0+\frac{1}{2}|\Omega|,~b^{*}=b_0+\frac{1}{2}\sum_{(i,j,t)\in\Omega}\left(y_{ijt}-\sum_{s=1}^{r}u_{is}v_{js}x_{ts}\right)^2.$$


#### - Sampling hyperparameters $\left(\boldsymbol{\mu}_{u},\Lambda_{u}\right)$

Draw

- $\Lambda_{u}\sim\mathcal{W}\left(W_u^{*},\nu_u^{*}\right)$
- $\boldsymbol{\mu}_{u}\sim\mathcal{N}\left(\boldsymbol{\mu}_{u}^{*},\left(\beta_u^{*}\Lambda_u\right)^{-1}\right)$

with following parameters:

$$\boldsymbol{\mu}_{u}^{*}=\frac{m\boldsymbol{\bar{u}}+\beta_0\boldsymbol{\mu}_0}{m+\beta_0},~\beta_u^{*}=m+\beta_0,~\nu_u^{*}=m+\nu_0,$$
$$\left(W_u^{*}\right)^{-1}=W_0^{-1}+mS_u+\frac{m\beta_0}{m+\beta_0}\left(\boldsymbol{\bar{u}}-\boldsymbol{\mu}_0\right)\left(\boldsymbol{\bar{u}}-\boldsymbol{\mu}_0\right)^T,$$
where $\boldsymbol{\bar{u}}=\sum_{i=1}^{m}\boldsymbol{u}_{i},~S_u=\frac{1}{m}\sum_{i=1}^{m}\left(\boldsymbol{u}_{i}-\boldsymbol{\bar{u}}\right)\left(\boldsymbol{u}_{i}-\boldsymbol{\bar{u}}\right)^T$.


#### - Sampling hyperparameters $\left(\boldsymbol{\mu}_{v},\Lambda_{v}\right)$

Draw

- $\Lambda_{v}\sim\mathcal{W}\left(W_v^{*},\nu_v^{*}\right)$
- $\boldsymbol{\mu}_{v}\sim\mathcal{N}\left(\boldsymbol{\mu}_{v}^{*},\left(\beta_v^{*}\Lambda_v\right)^{-1}\right)$

with following parameters:

$$\boldsymbol{\mu}_{v}^{*}=\frac{n\boldsymbol{\bar{v}}+\beta_0\boldsymbol{\mu}_0}{n+\beta_0},~\beta_v^{*}=n+\beta_0,~\nu_v^{*}=n+\nu_0,$$
$$\left(W_v^{*}\right)^{-1}=W_0^{-1}+nS_v+\frac{n\beta_0}{n+\beta_0}\left(\boldsymbol{\bar{v}}-\boldsymbol{\mu}_0\right)\left(\boldsymbol{\bar{v}}-\boldsymbol{\mu}_0\right)^T,$$
where $\boldsymbol{\bar{v}}=\sum_{j=1}^{n}\boldsymbol{v}_{j},~S_v=\frac{1}{n}\sum_{j=1}^{n}\left(\boldsymbol{v}_{j}-\boldsymbol{\bar{v}}\right)\left(\boldsymbol{v}_{j}-\boldsymbol{\bar{v}}\right)^T$.


#### - Sampling hyperparameters $\left(\boldsymbol{\mu}_{x},\Lambda_{x}\right)$

Draw

- $\Lambda_{x}\sim\mathcal{W}\left(W_x^{*},\nu_x^{*}\right)$
- $\boldsymbol{\mu}_{x}\sim\mathcal{N}\left(\boldsymbol{\mu}_{x}^{*},\left(\beta_x^{*}\Lambda_x\right)^{-1}\right)$

with following parameters:

$$\boldsymbol{\mu}_{x}^{*}=\frac{f\boldsymbol{\bar{x}}+\beta_0\boldsymbol{\mu}_0}{f+\beta_0},~\beta_x^{*}=f+\beta_0,~\nu_x^{*}=f+\nu_0,$$
$$\left(W_x^{*}\right)^{-1}=W_0^{-1}+fS_x+\frac{f\beta_0}{f+\beta_0}\left(\boldsymbol{\bar{x}}-\boldsymbol{\mu}_0\right)\left(\boldsymbol{\bar{x}}-\boldsymbol{\mu}_0\right)^T,$$
where $\boldsymbol{\bar{x}}=\sum_{t=1}^{f}\boldsymbol{x}_{t},~S_x=\frac{1}{f}\sum_{t=1}^{f}\left(\boldsymbol{x}_{t}-\boldsymbol{\bar{x}}\right)\left(\boldsymbol{x}_{t}-\boldsymbol{\bar{x}}\right)^T$.

### Define Performance Metrics

- **RMSE**
- **MAPE**

In [10]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

### Define BGCP with `Numpy`

In [16]:
def BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter):
    """Bayesian Gaussian CP (BGCP) decomposition."""
    
    dim = np.array(sparse_tensor.shape)
    rank = factor[0].shape[1]
    pos_train = np.where(sparse_tensor != 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    binary_tensor = np.zeros(dim)
    binary_tensor[pos_train] = 1
    
    beta0 = 1
    nu0 = rank
    mu0 = np.zeros((rank))
    W0 = np.eye(rank)
    tau = 1
    alpha = 1e-6
    beta = 1e-6
    
    factor_plus = []
    for k in range(len(dim)):
        factor_plus.append(np.zeros((dim[k], rank)))
    tensor_hat_plus = np.zeros(dim)
    for it in range(burnin_iter + gibbs_iter):
        for k in range(len(dim)):
            mat_bar = np.mean(factor[k], axis = 0)
            var_mu_hyper = (dim[k] * mat_bar + beta0 * mu0) / (dim[k] + beta0)
            var_W_hyper = inv(inv(W0) + cov_mat(factor[k]) + dim[k] * beta0 / (dim[k] + beta0)
                              * np.outer(mat_bar - mu0, mat_bar - mu0))
            var_Lambda_hyper = wishart(df = dim[k] + nu0, scale = var_W_hyper, seed = None).rvs()
            var_mu_hyper = mvnrnd(var_mu_hyper, inv((dim[k] + beta0) * var_Lambda_hyper))
            
            if k == 0:
                var1 = kr_prod(factor[k + 2], factor[k + 1]).T
            elif k == 1:
                var1 = kr_prod(factor[k + 1], factor[k - 1]).T
            else:
                var1 = kr_prod(factor[k - 1], factor[k - 2]).T
            var2 = kr_prod(var1, var1)
            var3 = (tau * np.matmul(var2, ten2mat(binary_tensor, k).T).reshape([rank, rank, dim[k]])
                    + np.dstack([var_Lambda_hyper] * dim[k]))
            var4 = (tau * np.matmul(var1, ten2mat(sparse_tensor, k).T)
                    + np.dstack([np.matmul(var_Lambda_hyper, var_mu_hyper)] * dim[k])[0, :, :])
            for i in range(dim[k]):
                var_Lambda = var3[:, :, i]
                inv_var_Lambda = inv((var_Lambda + var_Lambda.T) / 2)
                factor[k][i, :] = mvnrnd(np.matmul(inv_var_Lambda, var4[:, i]), inv_var_Lambda)
        tensor_hat = cp_combine(factor)
        var_alpha = alpha + 0.5 * sparse_tensor[pos_train].shape[0]
        var_beta = beta + 0.5 * np.sum((sparse_tensor - tensor_hat)[pos_train] ** 2)
        tau = np.random.gamma(var_alpha, 1 / var_beta)
        if it + 1 > burnin_iter:
            factor_plus = [factor_plus[k] + factor[k] for k in range(len(dim))]
            tensor_hat_plus += tensor_hat
        if (it + 1) % 1 == 0 and it < burnin_iter:
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        
    factor = [i / gibbs_iter for i in factor_plus]
    tensor_hat = tensor_hat_plus / gibbs_iter
    print('Final MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Final RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat, factor

### Experiments on PeMS-4W Data Set

In [29]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [30]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.0949634
RMSE: 6.76933

Iter: 2
MAPE: 0.0876329
RMSE: 6.38835

Iter: 3
MAPE: 0.0678421
RMSE: 5.29803

Iter: 4
MAPE: 0.0587958
RMSE: 4.75749

Iter: 5
MAPE: 0.0569541
RMSE: 4.66268

Iter: 6
MAPE: 0.0561762
RMSE: 4.62227

Iter: 7
MAPE: 0.0555884
RMSE: 4.59535

Iter: 8
MAPE: 0.0551568
RMSE: 4.57653

Iter: 9
MAPE: 0.0548511
RMSE: 4.56265

Iter: 10
MAPE: 0.0546031
RMSE: 4.55067

Iter: 11
MAPE: 0.0543885
RMSE: 4.53966

Iter: 12
MAPE: 0.0542222
RMSE: 4.52997

Iter: 13
MAPE: 0.0540605
RMSE: 4.52085

Iter: 14
MAPE: 0.0538849
RMSE: 4.51116

Iter: 15
MAPE: 0.0536898
RMSE: 4.501

Iter: 16
MAPE: 0.0535114
RMSE: 4.49203

Iter: 17
MAPE: 0.0533898
RMSE: 4.48495

Iter: 18
MAPE: 0.0533031
RMSE: 4.47943

Iter: 19
MAPE: 0.0532483
RMSE: 4.47528

Iter: 20
MAPE: 0.0532198
RMSE: 4.47185

Iter: 21
MAPE: 0.0531947
RMSE: 4.46898

Iter: 22
MAPE: 0.0531802
RMSE: 4.46629

Iter: 23
MAPE: 0.0531702
RMSE: 4.46384

Iter: 24
MAPE: 0.0531617
RMSE: 4.4616

Iter: 25
MAPE: 0.0531501
RMSE: 4.45925

Iter: 26
MAP

RMSE: 4.3647

Iter: 205
MAPE: 0.0515573
RMSE: 4.36423

Iter: 206
MAPE: 0.0515509
RMSE: 4.36392

Iter: 207
MAPE: 0.0515428
RMSE: 4.3635

Iter: 208
MAPE: 0.0515273
RMSE: 4.36308

Iter: 209
MAPE: 0.0515223
RMSE: 4.36275

Iter: 210
MAPE: 0.0515142
RMSE: 4.3624

Iter: 211
MAPE: 0.0515006
RMSE: 4.36202

Iter: 212
MAPE: 0.0514932
RMSE: 4.36168

Iter: 213
MAPE: 0.0514878
RMSE: 4.36133

Iter: 214
MAPE: 0.0514782
RMSE: 4.36096

Iter: 215
MAPE: 0.0514714
RMSE: 4.36057

Iter: 216
MAPE: 0.0514598
RMSE: 4.36028

Iter: 217
MAPE: 0.0514542
RMSE: 4.35996

Iter: 218
MAPE: 0.0514425
RMSE: 4.35955

Iter: 219
MAPE: 0.0514381
RMSE: 4.3594

Iter: 220
MAPE: 0.0514287
RMSE: 4.35894

Iter: 221
MAPE: 0.0514201
RMSE: 4.35864

Iter: 222
MAPE: 0.0514118
RMSE: 4.35841

Iter: 223
MAPE: 0.0514029
RMSE: 4.35814

Iter: 224
MAPE: 0.0514024
RMSE: 4.35793

Iter: 225
MAPE: 0.0513901
RMSE: 4.3576

Iter: 226
MAPE: 0.051386
RMSE: 4.35739

Iter: 227
MAPE: 0.051381
RMSE: 4.35715

Iter: 228
MAPE: 0.0513758
RMSE: 4.35683

Iter: 22

Iter: 406
MAPE: 0.0509973
RMSE: 4.33545

Iter: 407
MAPE: 0.0509995
RMSE: 4.33543

Iter: 408
MAPE: 0.0509953
RMSE: 4.33535

Iter: 409
MAPE: 0.0509985
RMSE: 4.33536

Iter: 410
MAPE: 0.0509931
RMSE: 4.33518

Iter: 411
MAPE: 0.0509909
RMSE: 4.3351

Iter: 412
MAPE: 0.0509892
RMSE: 4.33489

Iter: 413
MAPE: 0.0509849
RMSE: 4.33492

Iter: 414
MAPE: 0.0509837
RMSE: 4.3347

Iter: 415
MAPE: 0.0509813
RMSE: 4.3347

Iter: 416
MAPE: 0.0509775
RMSE: 4.33463

Iter: 417
MAPE: 0.050976
RMSE: 4.33464

Iter: 418
MAPE: 0.0509745
RMSE: 4.33456

Iter: 419
MAPE: 0.0509695
RMSE: 4.33435

Iter: 420
MAPE: 0.0509684
RMSE: 4.33421

Iter: 421
MAPE: 0.0509662
RMSE: 4.33419

Iter: 422
MAPE: 0.0509663
RMSE: 4.33411

Iter: 423
MAPE: 0.0509609
RMSE: 4.33404

Iter: 424
MAPE: 0.0509636
RMSE: 4.33396

Iter: 425
MAPE: 0.0509602
RMSE: 4.33375

Iter: 426
MAPE: 0.0509589
RMSE: 4.33384

Iter: 427
MAPE: 0.050953
RMSE: 4.33368

Iter: 428
MAPE: 0.0509495
RMSE: 4.33368

Iter: 429
MAPE: 0.0509509
RMSE: 4.33349

Iter: 430
MAPE: 0.050

In [31]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [32]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.101246
RMSE: 7.0858

Iter: 2
MAPE: 0.091642
RMSE: 6.6006

Iter: 3
MAPE: 0.0850313
RMSE: 6.15742

Iter: 4
MAPE: 0.0693774
RMSE: 5.27563

Iter: 5
MAPE: 0.0597596
RMSE: 4.80993

Iter: 6
MAPE: 0.0580581
RMSE: 4.72341

Iter: 7
MAPE: 0.0562992
RMSE: 4.63251

Iter: 8
MAPE: 0.0554516
RMSE: 4.58571

Iter: 9
MAPE: 0.0548281
RMSE: 4.54812

Iter: 10
MAPE: 0.0544079
RMSE: 4.52477

Iter: 11
MAPE: 0.0541741
RMSE: 4.51214

Iter: 12
MAPE: 0.054035
RMSE: 4.50449

Iter: 13
MAPE: 0.0539225
RMSE: 4.49823

Iter: 14
MAPE: 0.0538131
RMSE: 4.49258

Iter: 15
MAPE: 0.0537298
RMSE: 4.48785

Iter: 16
MAPE: 0.053656
RMSE: 4.4835

Iter: 17
MAPE: 0.053595
RMSE: 4.47985

Iter: 18
MAPE: 0.0535358
RMSE: 4.47652

Iter: 19
MAPE: 0.0534883
RMSE: 4.47346

Iter: 20
MAPE: 0.0534436
RMSE: 4.4708

Iter: 21
MAPE: 0.053406
RMSE: 4.46856

Iter: 22
MAPE: 0.053374
RMSE: 4.46643

Iter: 23
MAPE: 0.0533453
RMSE: 4.46416

Iter: 24
MAPE: 0.0533073
RMSE: 4.46243

Iter: 25
MAPE: 0.053283
RMSE: 4.46068

Iter: 26
MAPE: 0.0532

Iter: 205
MAPE: 0.0521224
RMSE: 4.39037

Iter: 206
MAPE: 0.0521181
RMSE: 4.39014

Iter: 207
MAPE: 0.0521005
RMSE: 4.38945

Iter: 208
MAPE: 0.0521046
RMSE: 4.38925

Iter: 209
MAPE: 0.0520935
RMSE: 4.38876

Iter: 210
MAPE: 0.0520914
RMSE: 4.38833

Iter: 211
MAPE: 0.0520847
RMSE: 4.38796

Iter: 212
MAPE: 0.0520748
RMSE: 4.38754

Iter: 213
MAPE: 0.0520666
RMSE: 4.38705

Iter: 214
MAPE: 0.0520548
RMSE: 4.3865

Iter: 215
MAPE: 0.0520549
RMSE: 4.38616

Iter: 216
MAPE: 0.0520466
RMSE: 4.38588

Iter: 217
MAPE: 0.0520427
RMSE: 4.3855

Iter: 218
MAPE: 0.0520349
RMSE: 4.38497

Iter: 219
MAPE: 0.0520176
RMSE: 4.38432

Iter: 220
MAPE: 0.0520079
RMSE: 4.38391

Iter: 221
MAPE: 0.0520055
RMSE: 4.38353

Iter: 222
MAPE: 0.0519954
RMSE: 4.38305

Iter: 223
MAPE: 0.0519914
RMSE: 4.38284

Iter: 224
MAPE: 0.0519838
RMSE: 4.38236

Iter: 225
MAPE: 0.0519776
RMSE: 4.38201

Iter: 226
MAPE: 0.0519638
RMSE: 4.38155

Iter: 227
MAPE: 0.0519652
RMSE: 4.3813

Iter: 228
MAPE: 0.0519505
RMSE: 4.3809

Iter: 229
MAPE: 0.05

Iter: 406
MAPE: 0.0512837
RMSE: 4.35494

Iter: 407
MAPE: 0.051276
RMSE: 4.35485

Iter: 408
MAPE: 0.0512863
RMSE: 4.35503

Iter: 409
MAPE: 0.0512868
RMSE: 4.35488

Iter: 410
MAPE: 0.0512783
RMSE: 4.35479

Iter: 411
MAPE: 0.0512791
RMSE: 4.35483

Iter: 412
MAPE: 0.0512714
RMSE: 4.3546

Iter: 413
MAPE: 0.0512672
RMSE: 4.35457

Iter: 414
MAPE: 0.0512684
RMSE: 4.35473

Iter: 415
MAPE: 0.051263
RMSE: 4.35463

Iter: 416
MAPE: 0.0512679
RMSE: 4.35443

Iter: 417
MAPE: 0.0512706
RMSE: 4.35452

Iter: 418
MAPE: 0.0512669
RMSE: 4.35444

Iter: 419
MAPE: 0.051264
RMSE: 4.35442

Iter: 420
MAPE: 0.0512618
RMSE: 4.35428

Iter: 421
MAPE: 0.0512666
RMSE: 4.35435

Iter: 422
MAPE: 0.0512529
RMSE: 4.35428

Iter: 423
MAPE: 0.0512607
RMSE: 4.3542

Iter: 424
MAPE: 0.0512541
RMSE: 4.35417

Iter: 425
MAPE: 0.0512534
RMSE: 4.35414

Iter: 426
MAPE: 0.0512587
RMSE: 4.35411

Iter: 427
MAPE: 0.0512521
RMSE: 4.35397

Iter: 428
MAPE: 0.051244
RMSE: 4.35395

Iter: 429
MAPE: 0.0512481
RMSE: 4.35392

Iter: 430
MAPE: 0.0512

In [33]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [34]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.163006
RMSE: 11.1177

Iter: 2
MAPE: 0.094242
RMSE: 6.79071

Iter: 3
MAPE: 0.0930078
RMSE: 6.74088

Iter: 4
MAPE: 0.092937
RMSE: 6.74712

Iter: 5
MAPE: 0.0877095
RMSE: 6.60198

Iter: 6
MAPE: 0.077253
RMSE: 6.09712

Iter: 7
MAPE: 0.0744238
RMSE: 6.12491

Iter: 8
MAPE: 0.0682376
RMSE: 5.57939

Iter: 9
MAPE: 0.0658328
RMSE: 5.43518

Iter: 10
MAPE: 0.0638786
RMSE: 5.24476

Iter: 11
MAPE: 0.0616119
RMSE: 5.03982

Iter: 12
MAPE: 0.0597629
RMSE: 4.90063

Iter: 13
MAPE: 0.0588246
RMSE: 4.83616

Iter: 14
MAPE: 0.0584407
RMSE: 4.81146

Iter: 15
MAPE: 0.0582492
RMSE: 4.80123

Iter: 16
MAPE: 0.058147
RMSE: 4.79594

Iter: 17
MAPE: 0.0580756
RMSE: 4.79418

Iter: 18
MAPE: 0.0580269
RMSE: 4.79251

Iter: 19
MAPE: 0.0580038
RMSE: 4.79275

Iter: 20
MAPE: 0.0579697
RMSE: 4.79198

Iter: 21
MAPE: 0.0579654
RMSE: 4.79373

Iter: 22
MAPE: 0.0579572
RMSE: 4.79347

Iter: 23
MAPE: 0.0579334
RMSE: 4.79367

Iter: 24
MAPE: 0.057914
RMSE: 4.79424

Iter: 25
MAPE: 0.0579191
RMSE: 4.7946

Iter: 26
MAPE: 0

RMSE: 4.64537

Iter: 205
MAPE: 0.055052
RMSE: 4.64506

Iter: 206
MAPE: 0.0550485
RMSE: 4.64436

Iter: 207
MAPE: 0.0550484
RMSE: 4.6444

Iter: 208
MAPE: 0.0550409
RMSE: 4.64443

Iter: 209
MAPE: 0.0550521
RMSE: 4.6451

Iter: 210
MAPE: 0.0550357
RMSE: 4.64412

Iter: 211
MAPE: 0.0550279
RMSE: 4.64368

Iter: 212
MAPE: 0.0550234
RMSE: 4.64392

Iter: 213
MAPE: 0.0550198
RMSE: 4.64352

Iter: 214
MAPE: 0.0550199
RMSE: 4.64396

Iter: 215
MAPE: 0.0550125
RMSE: 4.64347

Iter: 216
MAPE: 0.0550188
RMSE: 4.64392

Iter: 217
MAPE: 0.0549982
RMSE: 4.64312

Iter: 218
MAPE: 0.0549955
RMSE: 4.6425

Iter: 219
MAPE: 0.0549893
RMSE: 4.643

Iter: 220
MAPE: 0.0549891
RMSE: 4.64218

Iter: 221
MAPE: 0.0549852
RMSE: 4.64249

Iter: 222
MAPE: 0.0549807
RMSE: 4.64201

Iter: 223
MAPE: 0.0549793
RMSE: 4.64231

Iter: 224
MAPE: 0.0549753
RMSE: 4.64225

Iter: 225
MAPE: 0.0549598
RMSE: 4.64176

Iter: 226
MAPE: 0.0549678
RMSE: 4.64174

Iter: 227
MAPE: 0.0549458
RMSE: 4.64108

Iter: 228
MAPE: 0.0549559
RMSE: 4.64124

Iter: 2

Iter: 406
MAPE: 0.054617
RMSE: 4.6283

Iter: 407
MAPE: 0.0546168
RMSE: 4.62761

Iter: 408
MAPE: 0.0546244
RMSE: 4.62848

Iter: 409
MAPE: 0.0546269
RMSE: 4.62806

Iter: 410
MAPE: 0.054624
RMSE: 4.6285

Iter: 411
MAPE: 0.0546219
RMSE: 4.62836

Iter: 412
MAPE: 0.0546223
RMSE: 4.62864

Iter: 413
MAPE: 0.0546182
RMSE: 4.62839

Iter: 414
MAPE: 0.0546067
RMSE: 4.628

Iter: 415
MAPE: 0.0546199
RMSE: 4.62796

Iter: 416
MAPE: 0.0546151
RMSE: 4.62781

Iter: 417
MAPE: 0.0546112
RMSE: 4.62764

Iter: 418
MAPE: 0.0546156
RMSE: 4.62796

Iter: 419
MAPE: 0.0546278
RMSE: 4.62852

Iter: 420
MAPE: 0.0546073
RMSE: 4.62736

Iter: 421
MAPE: 0.0546136
RMSE: 4.62759

Iter: 422
MAPE: 0.0546029
RMSE: 4.62751

Iter: 423
MAPE: 0.0546239
RMSE: 4.62843

Iter: 424
MAPE: 0.0546051
RMSE: 4.6278

Iter: 425
MAPE: 0.0546205
RMSE: 4.62785

Iter: 426
MAPE: 0.0546043
RMSE: 4.62768

Iter: 427
MAPE: 0.0546201
RMSE: 4.62854

Iter: 428
MAPE: 0.0546131
RMSE: 4.62785

Iter: 429
MAPE: 0.0546257
RMSE: 4.62841

Iter: 430
MAPE: 0.05461

In [35]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [36]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.290401
RMSE: 20.7797

Iter: 2
MAPE: 0.119892
RMSE: 8.33394

Iter: 3
MAPE: 0.137178
RMSE: 10.0086

Iter: 4
MAPE: 0.223995
RMSE: 19.0491

Iter: 5
MAPE: 0.412034
RMSE: 37.3227

Iter: 6
MAPE: 0.57499
RMSE: 54.5098

Iter: 7
MAPE: 0.668557
RMSE: 66.0398

Iter: 8
MAPE: 0.719046
RMSE: 72.9481

Iter: 9
MAPE: 0.749703
RMSE: 77.2778

Iter: 10
MAPE: 0.768061
RMSE: 79.9321

Iter: 11
MAPE: 0.779827
RMSE: 81.8043

Iter: 12
MAPE: 0.786705
RMSE: 83.075

Iter: 13
MAPE: 0.785599
RMSE: 83.329

Iter: 14
MAPE: 0.779022
RMSE: 83.0007

Iter: 15
MAPE: 0.767257
RMSE: 82.069

Iter: 16
MAPE: 0.753681
RMSE: 80.9756

Iter: 17
MAPE: 0.733821
RMSE: 79.245

Iter: 18
MAPE: 0.712219
RMSE: 77.1557

Iter: 19
MAPE: 0.688261
RMSE: 75.0499

Iter: 20
MAPE: 0.664144
RMSE: 72.8063

Iter: 21
MAPE: 0.64208
RMSE: 70.8757

Iter: 22
MAPE: 0.62003
RMSE: 68.7097

Iter: 23
MAPE: 0.602616
RMSE: 67.2438

Iter: 24
MAPE: 0.585788
RMSE: 65.6275

Iter: 25
MAPE: 0.570785
RMSE: 64.1327

Iter: 26
MAPE: 0.561175
RMSE: 63.193

Ite

Iter: 207
MAPE: 0.0720821
RMSE: 6.80862

Iter: 208
MAPE: 0.0721317
RMSE: 6.81578

Iter: 209
MAPE: 0.072128
RMSE: 6.81198

Iter: 210
MAPE: 0.0721719
RMSE: 6.82666

Iter: 211
MAPE: 0.0721725
RMSE: 6.83418

Iter: 212
MAPE: 0.0721182
RMSE: 6.83129

Iter: 213
MAPE: 0.0720915
RMSE: 6.82896

Iter: 214
MAPE: 0.0721307
RMSE: 6.84287

Iter: 215
MAPE: 0.0721604
RMSE: 6.84575

Iter: 216
MAPE: 0.0722147
RMSE: 6.85511

Iter: 217
MAPE: 0.072313
RMSE: 6.87783

Iter: 218
MAPE: 0.0722593
RMSE: 6.87201

Iter: 219
MAPE: 0.0723663
RMSE: 6.89234

Iter: 220
MAPE: 0.0723525
RMSE: 6.89765

Iter: 221
MAPE: 0.0722804
RMSE: 6.87971

Iter: 222
MAPE: 0.0723233
RMSE: 6.89828

Iter: 223
MAPE: 0.0723013
RMSE: 6.89789

Iter: 224
MAPE: 0.072334
RMSE: 6.90558

Iter: 225
MAPE: 0.0723555
RMSE: 6.92533

Iter: 226
MAPE: 0.0723113
RMSE: 6.92444

Iter: 227
MAPE: 0.0723748
RMSE: 6.93848

Iter: 228
MAPE: 0.0723968
RMSE: 6.93809

Iter: 229
MAPE: 0.0724277
RMSE: 6.94978

Iter: 230
MAPE: 0.0725006
RMSE: 6.95834

Iter: 231
MAPE: 0.0

Iter: 408
MAPE: 0.0857925
RMSE: 12.0858

Iter: 409
MAPE: 0.0859431
RMSE: 12.1305

Iter: 410
MAPE: 0.0859613
RMSE: 12.1641

Iter: 411
MAPE: 0.0862777
RMSE: 12.2919

Iter: 412
MAPE: 0.0864801
RMSE: 12.3589

Iter: 413
MAPE: 0.0865979
RMSE: 12.4167

Iter: 414
MAPE: 0.0868547
RMSE: 12.5051

Iter: 415
MAPE: 0.0869622
RMSE: 12.5664

Iter: 416
MAPE: 0.0870728
RMSE: 12.6929

Iter: 417
MAPE: 0.087106
RMSE: 12.6743

Iter: 418
MAPE: 0.0872651
RMSE: 12.7347

Iter: 419
MAPE: 0.0873823
RMSE: 12.7908

Iter: 420
MAPE: 0.0875523
RMSE: 12.8515

Iter: 421
MAPE: 0.087723
RMSE: 12.932

Iter: 422
MAPE: 0.0877227
RMSE: 12.941

Iter: 423
MAPE: 0.0877488
RMSE: 12.9652

Iter: 424
MAPE: 0.0878075
RMSE: 12.9924

Iter: 425
MAPE: 0.0878754
RMSE: 13.0292

Iter: 426
MAPE: 0.0880735
RMSE: 13.1325

Iter: 427
MAPE: 0.0880673
RMSE: 13.1458

Iter: 428
MAPE: 0.0880462
RMSE: 13.1571

Iter: 429
MAPE: 0.0881365
RMSE: 13.1525

Iter: 430
MAPE: 0.08814
RMSE: 13.1588

Iter: 431
MAPE: 0.088087
RMSE: 13.1704

Iter: 432
MAPE: 0.08828

### Experiments on PeMS-8W Data Set

In [37]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [38]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.0994997
RMSE: 7.00634

Iter: 2
MAPE: 0.0919107
RMSE: 6.57814

Iter: 3
MAPE: 0.0739557
RMSE: 5.68117

Iter: 4
MAPE: 0.0644607
RMSE: 5.10444

Iter: 5
MAPE: 0.0625488
RMSE: 5.01104

Iter: 6
MAPE: 0.062079
RMSE: 4.98596

Iter: 7
MAPE: 0.0612345
RMSE: 4.94266

Iter: 8
MAPE: 0.0596756
RMSE: 4.86851

Iter: 9
MAPE: 0.0583558
RMSE: 4.8081

Iter: 10
MAPE: 0.0577808
RMSE: 4.78139

Iter: 11
MAPE: 0.0575438
RMSE: 4.76816

Iter: 12
MAPE: 0.0574053
RMSE: 4.75933

Iter: 13
MAPE: 0.0573162
RMSE: 4.75337

Iter: 14
MAPE: 0.057255
RMSE: 4.74919

Iter: 15
MAPE: 0.0572204
RMSE: 4.74595

Iter: 16
MAPE: 0.0571919
RMSE: 4.74336

Iter: 17
MAPE: 0.0571669
RMSE: 4.74103

Iter: 18
MAPE: 0.0571438
RMSE: 4.73886

Iter: 19
MAPE: 0.0571173
RMSE: 4.73669

Iter: 20
MAPE: 0.0570943
RMSE: 4.7346

Iter: 21
MAPE: 0.0570612
RMSE: 4.73228

Iter: 22
MAPE: 0.0570242
RMSE: 4.72985

Iter: 23
MAPE: 0.0569773
RMSE: 4.72726

Iter: 24
MAPE: 0.0569242
RMSE: 4.72438

Iter: 25
MAPE: 0.0568638
RMSE: 4.72147

Iter: 26
MAPE

Iter: 205
MAPE: 0.0547313
RMSE: 4.60986

Iter: 206
MAPE: 0.0547352
RMSE: 4.60988

Iter: 207
MAPE: 0.0547286
RMSE: 4.60972

Iter: 208
MAPE: 0.0547293
RMSE: 4.60978

Iter: 209
MAPE: 0.0547295
RMSE: 4.60969

Iter: 210
MAPE: 0.0547309
RMSE: 4.60968

Iter: 211
MAPE: 0.0547309
RMSE: 4.60966

Iter: 212
MAPE: 0.0547289
RMSE: 4.60966

Iter: 213
MAPE: 0.0547272
RMSE: 4.60958

Iter: 214
MAPE: 0.0547275
RMSE: 4.60958

Iter: 215
MAPE: 0.0547274
RMSE: 4.60959

Iter: 216
MAPE: 0.0547269
RMSE: 4.60955

Iter: 217
MAPE: 0.0547249
RMSE: 4.60951

Iter: 218
MAPE: 0.0547269
RMSE: 4.60945

Iter: 219
MAPE: 0.0547217
RMSE: 4.60943

Iter: 220
MAPE: 0.054723
RMSE: 4.60935

Iter: 221
MAPE: 0.0547223
RMSE: 4.60937

Iter: 222
MAPE: 0.0547216
RMSE: 4.6093

Iter: 223
MAPE: 0.0547203
RMSE: 4.60935

Iter: 224
MAPE: 0.0547215
RMSE: 4.60928

Iter: 225
MAPE: 0.0547192
RMSE: 4.60914

Iter: 226
MAPE: 0.0547241
RMSE: 4.60921

Iter: 227
MAPE: 0.0547178
RMSE: 4.6091

Iter: 228
MAPE: 0.0547206
RMSE: 4.6091

Iter: 229
MAPE: 0.05

MAPE: 0.0546312
RMSE: 4.60286

Iter: 407
MAPE: 0.0546291
RMSE: 4.60281

Iter: 408
MAPE: 0.0546289
RMSE: 4.60272

Iter: 409
MAPE: 0.0546257
RMSE: 4.60268

Iter: 410
MAPE: 0.0546276
RMSE: 4.60261

Iter: 411
MAPE: 0.0546251
RMSE: 4.60262

Iter: 412
MAPE: 0.0546267
RMSE: 4.60259

Iter: 413
MAPE: 0.0546266
RMSE: 4.60256

Iter: 414
MAPE: 0.0546243
RMSE: 4.6025

Iter: 415
MAPE: 0.0546224
RMSE: 4.60248

Iter: 416
MAPE: 0.0546262
RMSE: 4.60242

Iter: 417
MAPE: 0.0546193
RMSE: 4.60239

Iter: 418
MAPE: 0.0546213
RMSE: 4.60239

Iter: 419
MAPE: 0.0546227
RMSE: 4.60236

Iter: 420
MAPE: 0.0546202
RMSE: 4.60229

Iter: 421
MAPE: 0.0546226
RMSE: 4.60235

Iter: 422
MAPE: 0.0546159
RMSE: 4.60224

Iter: 423
MAPE: 0.05462
RMSE: 4.60226

Iter: 424
MAPE: 0.0546173
RMSE: 4.6022

Iter: 425
MAPE: 0.0546192
RMSE: 4.60215

Iter: 426
MAPE: 0.0546147
RMSE: 4.60212

Iter: 427
MAPE: 0.054615
RMSE: 4.60208

Iter: 428
MAPE: 0.0546134
RMSE: 4.60196

Iter: 429
MAPE: 0.0546131
RMSE: 4.60198

Iter: 430
MAPE: 0.0546133
RMSE:

In [39]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [40]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.102315
RMSE: 7.14052

Iter: 2
MAPE: 0.0954265
RMSE: 6.78927

Iter: 3
MAPE: 0.0743278
RMSE: 5.61688

Iter: 4
MAPE: 0.0626109
RMSE: 5.01377

Iter: 5
MAPE: 0.0613128
RMSE: 4.94279

Iter: 6
MAPE: 0.0607465
RMSE: 4.914

Iter: 7
MAPE: 0.0601979
RMSE: 4.88546

Iter: 8
MAPE: 0.0592711
RMSE: 4.84293

Iter: 9
MAPE: 0.0581776
RMSE: 4.79533

Iter: 10
MAPE: 0.0577878
RMSE: 4.77686

Iter: 11
MAPE: 0.0576746
RMSE: 4.76882

Iter: 12
MAPE: 0.0576202
RMSE: 4.76346

Iter: 13
MAPE: 0.0575712
RMSE: 4.75978

Iter: 14
MAPE: 0.0575388
RMSE: 4.75712

Iter: 15
MAPE: 0.0575105
RMSE: 4.75514

Iter: 16
MAPE: 0.0574791
RMSE: 4.75343

Iter: 17
MAPE: 0.0574698
RMSE: 4.75205

Iter: 18
MAPE: 0.0574543
RMSE: 4.75066

Iter: 19
MAPE: 0.0574303
RMSE: 4.74939

Iter: 20
MAPE: 0.0574213
RMSE: 4.74821

Iter: 21
MAPE: 0.0573993
RMSE: 4.74706

Iter: 22
MAPE: 0.057382
RMSE: 4.74591

Iter: 23
MAPE: 0.0573755
RMSE: 4.74489

Iter: 24
MAPE: 0.0573602
RMSE: 4.74384

Iter: 25
MAPE: 0.0573458
RMSE: 4.74281

Iter: 26
MAPE

RMSE: 4.60558

Iter: 205
MAPE: 0.0546435
RMSE: 4.60548

Iter: 206
MAPE: 0.0546466
RMSE: 4.60543

Iter: 207
MAPE: 0.0546408
RMSE: 4.60551

Iter: 208
MAPE: 0.0546383
RMSE: 4.60541

Iter: 209
MAPE: 0.0546475
RMSE: 4.60537

Iter: 210
MAPE: 0.0546406
RMSE: 4.60536

Iter: 211
MAPE: 0.0546368
RMSE: 4.6053

Iter: 212
MAPE: 0.0546366
RMSE: 4.60538

Iter: 213
MAPE: 0.0546354
RMSE: 4.60528

Iter: 214
MAPE: 0.054634
RMSE: 4.60519

Iter: 215
MAPE: 0.0546344
RMSE: 4.60527

Iter: 216
MAPE: 0.0546378
RMSE: 4.60522

Iter: 217
MAPE: 0.0546349
RMSE: 4.60518

Iter: 218
MAPE: 0.0546331
RMSE: 4.60514

Iter: 219
MAPE: 0.054633
RMSE: 4.60511

Iter: 220
MAPE: 0.0546259
RMSE: 4.60512

Iter: 221
MAPE: 0.0546335
RMSE: 4.60509

Iter: 222
MAPE: 0.054627
RMSE: 4.60499

Iter: 223
MAPE: 0.05463
RMSE: 4.60512

Iter: 224
MAPE: 0.0546244
RMSE: 4.60498

Iter: 225
MAPE: 0.0546237
RMSE: 4.60494

Iter: 226
MAPE: 0.0546331
RMSE: 4.60491

Iter: 227
MAPE: 0.0546237
RMSE: 4.60488

Iter: 228
MAPE: 0.0546215
RMSE: 4.60484

Iter: 2

RMSE: 4.60172

Iter: 406
MAPE: 0.054532
RMSE: 4.60153

Iter: 407
MAPE: 0.0545355
RMSE: 4.60158

Iter: 408
MAPE: 0.0545311
RMSE: 4.60158

Iter: 409
MAPE: 0.0545252
RMSE: 4.60151

Iter: 410
MAPE: 0.0545295
RMSE: 4.60153

Iter: 411
MAPE: 0.0545348
RMSE: 4.60147

Iter: 412
MAPE: 0.0545358
RMSE: 4.60145

Iter: 413
MAPE: 0.0545309
RMSE: 4.60145

Iter: 414
MAPE: 0.0545334
RMSE: 4.60151

Iter: 415
MAPE: 0.0545295
RMSE: 4.60145

Iter: 416
MAPE: 0.0545301
RMSE: 4.60145

Iter: 417
MAPE: 0.0545299
RMSE: 4.60153

Iter: 418
MAPE: 0.0545321
RMSE: 4.6014

Iter: 419
MAPE: 0.0545343
RMSE: 4.60136

Iter: 420
MAPE: 0.0545263
RMSE: 4.60136

Iter: 421
MAPE: 0.0545294
RMSE: 4.60135

Iter: 422
MAPE: 0.0545304
RMSE: 4.60129

Iter: 423
MAPE: 0.0545278
RMSE: 4.60142

Iter: 424
MAPE: 0.0545347
RMSE: 4.60128

Iter: 425
MAPE: 0.0545319
RMSE: 4.60135

Iter: 426
MAPE: 0.0545227
RMSE: 4.60128

Iter: 427
MAPE: 0.0545232
RMSE: 4.6012

Iter: 428
MAPE: 0.0545246
RMSE: 4.60126

Iter: 429
MAPE: 0.0545282
RMSE: 4.60117

Iter

In [41]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [42]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.13752
RMSE: 9.14879

Iter: 2
MAPE: 0.0974627
RMSE: 6.88691

Iter: 3
MAPE: 0.0795954
RMSE: 6.00472

Iter: 4
MAPE: 0.0670381
RMSE: 5.32762

Iter: 5
MAPE: 0.0632933
RMSE: 5.14711

Iter: 6
MAPE: 0.0618829
RMSE: 5.04886

Iter: 7
MAPE: 0.0613731
RMSE: 5.0035

Iter: 8
MAPE: 0.0610477
RMSE: 4.97911

Iter: 9
MAPE: 0.060725
RMSE: 4.95666

Iter: 10
MAPE: 0.0602922
RMSE: 4.92928

Iter: 11
MAPE: 0.0599514
RMSE: 4.91014

Iter: 12
MAPE: 0.059738
RMSE: 4.89907

Iter: 13
MAPE: 0.0595735
RMSE: 4.89167

Iter: 14
MAPE: 0.0594166
RMSE: 4.88445

Iter: 15
MAPE: 0.0592817
RMSE: 4.87865

Iter: 16
MAPE: 0.0591704
RMSE: 4.87241

Iter: 17
MAPE: 0.059065
RMSE: 4.86607

Iter: 18
MAPE: 0.0589552
RMSE: 4.85763

Iter: 19
MAPE: 0.0588253
RMSE: 4.84833

Iter: 20
MAPE: 0.0586956
RMSE: 4.8395

Iter: 21
MAPE: 0.0585958
RMSE: 4.83328

Iter: 22
MAPE: 0.058518
RMSE: 4.82934

Iter: 23
MAPE: 0.0584601
RMSE: 4.82645

Iter: 24
MAPE: 0.0584261
RMSE: 4.82488

Iter: 25
MAPE: 0.0583815
RMSE: 4.82326

Iter: 26
MAPE: 0.

RMSE: 4.70449

Iter: 205
MAPE: 0.0559462
RMSE: 4.70439

Iter: 206
MAPE: 0.05594
RMSE: 4.70421

Iter: 207
MAPE: 0.0559355
RMSE: 4.70383

Iter: 208
MAPE: 0.0559318
RMSE: 4.70369

Iter: 209
MAPE: 0.0559173
RMSE: 4.70316

Iter: 210
MAPE: 0.0559136
RMSE: 4.70302

Iter: 211
MAPE: 0.0558972
RMSE: 4.70249

Iter: 212
MAPE: 0.05589
RMSE: 4.70231

Iter: 213
MAPE: 0.0558821
RMSE: 4.70183

Iter: 214
MAPE: 0.0558774
RMSE: 4.70184

Iter: 215
MAPE: 0.0558684
RMSE: 4.70159

Iter: 216
MAPE: 0.0558579
RMSE: 4.70133

Iter: 217
MAPE: 0.055846
RMSE: 4.70062

Iter: 218
MAPE: 0.0558406
RMSE: 4.70033

Iter: 219
MAPE: 0.0558314
RMSE: 4.70024

Iter: 220
MAPE: 0.0558185
RMSE: 4.69972

Iter: 221
MAPE: 0.0558151
RMSE: 4.69957

Iter: 222
MAPE: 0.0558068
RMSE: 4.69927

Iter: 223
MAPE: 0.0557944
RMSE: 4.69873

Iter: 224
MAPE: 0.0557843
RMSE: 4.69863

Iter: 225
MAPE: 0.0557702
RMSE: 4.69799

Iter: 226
MAPE: 0.0557607
RMSE: 4.69779

Iter: 227
MAPE: 0.0557548
RMSE: 4.69799

Iter: 228
MAPE: 0.0557428
RMSE: 4.69722

Iter: 

RMSE: 4.67862

Iter: 406
MAPE: 0.0553052
RMSE: 4.67894

Iter: 407
MAPE: 0.0553055
RMSE: 4.67921

Iter: 408
MAPE: 0.055302
RMSE: 4.67888

Iter: 409
MAPE: 0.0553017
RMSE: 4.67858

Iter: 410
MAPE: 0.0553037
RMSE: 4.67885

Iter: 411
MAPE: 0.0552983
RMSE: 4.6788

Iter: 412
MAPE: 0.0552989
RMSE: 4.6787

Iter: 413
MAPE: 0.0552978
RMSE: 4.67856

Iter: 414
MAPE: 0.0553012
RMSE: 4.67881

Iter: 415
MAPE: 0.0552967
RMSE: 4.6785

Iter: 416
MAPE: 0.0552921
RMSE: 4.67837

Iter: 417
MAPE: 0.055294
RMSE: 4.67852

Iter: 418
MAPE: 0.0552959
RMSE: 4.67849

Iter: 419
MAPE: 0.0552951
RMSE: 4.67841

Iter: 420
MAPE: 0.055293
RMSE: 4.67849

Iter: 421
MAPE: 0.0552963
RMSE: 4.67834

Iter: 422
MAPE: 0.0552949
RMSE: 4.67846

Iter: 423
MAPE: 0.0552919
RMSE: 4.67842

Iter: 424
MAPE: 0.0552929
RMSE: 4.67837

Iter: 425
MAPE: 0.0552972
RMSE: 4.67855

Iter: 426
MAPE: 0.0552938
RMSE: 4.67834

Iter: 427
MAPE: 0.0552953
RMSE: 4.67854

Iter: 428
MAPE: 0.0552951
RMSE: 4.67833

Iter: 429
MAPE: 0.0552953
RMSE: 4.67858

Iter: 4

In [43]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [44]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.rand(dim[k], rank))
burnin_iter = 500
gibbs_iter = 100
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 1
MAPE: 0.215935
RMSE: 14.9267

Iter: 2
MAPE: 0.10404
RMSE: 7.28536

Iter: 3
MAPE: 0.104076
RMSE: 7.34831

Iter: 4
MAPE: 0.104313
RMSE: 7.39654

Iter: 5
MAPE: 0.0982493
RMSE: 7.2722

Iter: 6
MAPE: 0.0849656
RMSE: 6.57065

Iter: 7
MAPE: 0.0769651
RMSE: 6.05283

Iter: 8
MAPE: 0.0725771
RMSE: 5.80483

Iter: 9
MAPE: 0.0690631
RMSE: 5.58657

Iter: 10
MAPE: 0.067081
RMSE: 5.46392

Iter: 11
MAPE: 0.0659912
RMSE: 5.39787

Iter: 12
MAPE: 0.0652073
RMSE: 5.34554

Iter: 13
MAPE: 0.0646004
RMSE: 5.30161

Iter: 14
MAPE: 0.0640498
RMSE: 5.26106

Iter: 15
MAPE: 0.0635593
RMSE: 5.22546

Iter: 16
MAPE: 0.0631555
RMSE: 5.19575

Iter: 17
MAPE: 0.0628192
RMSE: 5.17305

Iter: 18
MAPE: 0.0625643
RMSE: 5.15695

Iter: 19
MAPE: 0.0623573
RMSE: 5.1445

Iter: 20
MAPE: 0.0621774
RMSE: 5.1343

Iter: 21
MAPE: 0.0620152
RMSE: 5.12646

Iter: 22
MAPE: 0.0618695
RMSE: 5.11909

Iter: 23
MAPE: 0.0617196
RMSE: 5.11063

Iter: 24
MAPE: 0.0615756
RMSE: 5.10161

Iter: 25
MAPE: 0.0614321
RMSE: 5.09252

Iter: 26
MAPE: 0.0

Iter: 205
MAPE: 0.057714
RMSE: 4.84665

Iter: 206
MAPE: 0.0577096
RMSE: 4.84661

Iter: 207
MAPE: 0.0577157
RMSE: 4.84676

Iter: 208
MAPE: 0.0577118
RMSE: 4.84595

Iter: 209
MAPE: 0.0577078
RMSE: 4.84634

Iter: 210
MAPE: 0.0577093
RMSE: 4.84615

Iter: 211
MAPE: 0.0577089
RMSE: 4.8468

Iter: 212
MAPE: 0.0576988
RMSE: 4.84621

Iter: 213
MAPE: 0.0576968
RMSE: 4.84613

Iter: 214
MAPE: 0.0577026
RMSE: 4.84648

Iter: 215
MAPE: 0.0576967
RMSE: 4.84656

Iter: 216
MAPE: 0.0576974
RMSE: 4.84656

Iter: 217
MAPE: 0.0576909
RMSE: 4.84636

Iter: 218
MAPE: 0.0577031
RMSE: 4.84694

Iter: 219
MAPE: 0.0576984
RMSE: 4.84716

Iter: 220
MAPE: 0.0576997
RMSE: 4.84682

Iter: 221
MAPE: 0.0576979
RMSE: 4.84713

Iter: 222
MAPE: 0.0576856
RMSE: 4.8466

Iter: 223
MAPE: 0.0576889
RMSE: 4.84645

Iter: 224
MAPE: 0.0576813
RMSE: 4.84672

Iter: 225
MAPE: 0.0576811
RMSE: 4.84701

Iter: 226
MAPE: 0.0576859
RMSE: 4.84709

Iter: 227
MAPE: 0.0576816
RMSE: 4.84707

Iter: 228
MAPE: 0.0576963
RMSE: 4.84771

Iter: 229
MAPE: 0.0

RMSE: 4.84681

Iter: 406
MAPE: 0.0575281
RMSE: 4.84703

Iter: 407
MAPE: 0.0575172
RMSE: 4.84674

Iter: 408
MAPE: 0.057507
RMSE: 4.84631

Iter: 409
MAPE: 0.0575213
RMSE: 4.84664

Iter: 410
MAPE: 0.0575171
RMSE: 4.84667

Iter: 411
MAPE: 0.0575076
RMSE: 4.84616

Iter: 412
MAPE: 0.0575066
RMSE: 4.84558

Iter: 413
MAPE: 0.0575142
RMSE: 4.84645

Iter: 414
MAPE: 0.0575104
RMSE: 4.84617

Iter: 415
MAPE: 0.0575045
RMSE: 4.84598

Iter: 416
MAPE: 0.0575019
RMSE: 4.84646

Iter: 417
MAPE: 0.0574902
RMSE: 4.84575

Iter: 418
MAPE: 0.0575073
RMSE: 4.84621

Iter: 419
MAPE: 0.0574989
RMSE: 4.84565

Iter: 420
MAPE: 0.0574954
RMSE: 4.8455

Iter: 421
MAPE: 0.0574933
RMSE: 4.84539

Iter: 422
MAPE: 0.0574936
RMSE: 4.84564

Iter: 423
MAPE: 0.0574883
RMSE: 4.84504

Iter: 424
MAPE: 0.0574874
RMSE: 4.84539

Iter: 425
MAPE: 0.0574969
RMSE: 4.84553

Iter: 426
MAPE: 0.0574887
RMSE: 4.8455

Iter: 427
MAPE: 0.0574849
RMSE: 4.84549

Iter: 428
MAPE: 0.05747
RMSE: 4.84446

Iter: 429
MAPE: 0.0574864
RMSE: 4.84559

Iter: 

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>